In [1]:
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 02:26:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 02:26:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
consumer_fraud_sdf = read_file(spark, 'consumer_fraud.parquet', '../data/curated/')
consumer_fraud_sdf.count()

|> Loading File...


|> Loading Finished!
-RECORD 0------------------------------
 user_id           | 3188              
 order_datetime    | 2021-11-26        
 consumer_id       | 404921            
 fraud_probability | 9.611722331189984 
only showing top 1 row



7994010

In [5]:
consumer_fraud_sdf.show(5)

+-------+--------------+-----------+-----------------+
|user_id|order_datetime|consumer_id|fraud_probability|
+-------+--------------+-----------+-----------------+
|   3188|    2021-11-26|     404921|9.611722331189984|
|  21208|    2021-11-26|     111788|9.611722331189984|
|  18693|    2021-11-26|     188626|9.611722331189984|
|   9704|    2021-11-26|     365098|9.611722331189984|
|  15686|    2021-11-26|    1346536|9.611722331189984|
+-------+--------------+-----------+-----------------+
only showing top 5 rows



In [6]:
merchant_fraud_sdf = read_file(spark, 'merchants_fraud.parquet', '../data/curated/')
merchant_fraud_sdf.count()

|> Loading File...
|> Loading Finished!
-RECORD 0------------------------------
 merchant_abn      | 10023283211       
 order_datetime    | 2021-03-01        
 fraud_probability | 30.57903215900633 
only showing top 1 row



1204541

In [7]:
merchant_fraud_sdf.show(5)

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 10023283211|    2021-03-01| 30.57903215900633|
| 10023283211|    2021-03-05| 30.57903215900633|
| 10023283211|    2021-03-06|29.563080709651327|
| 10023283211|    2021-03-15| 80.80054474543395|
| 10023283211|    2021-03-17| 30.57903215900633|
+------------+--------------+------------------+
only showing top 5 rows



### Join with transaction data

In [8]:
trans_sdf = read_file(spark, 'changed_data', '../data/curated/')

|> Loading File...


|> Loading Finished!
-RECORD 0-------------------------------------------------------------------------------------------
 user_id         | 9521                                                                             
 merchant_abn    | 10023283211                                                                      
 dollar_value    | 86.95574860858353                                                                
 order_id        | c6c473f9-e796-4c1c-b952-4df1c93f3b4a                                             
 tags            | furniture, home furnishings and equipment shops, and manufacturers, except ap... 
 take_rate       | 0.18                                                                             
 type            | e                                                                                
 postcode        | 4753                                                                             
 consumer_id     | 838133                                             

In [9]:
trans_sdf = trans_sdf.select('order_id', 'order_datetime', 'user_id', 'merchant_abn')

In [10]:
trans_sdf.show(5)

+--------------------+--------------+-------+------------+
|            order_id|order_datetime|user_id|merchant_abn|
+--------------------+--------------+-------+------------+
|c6c473f9-e796-4c1...|    2021-11-26|   9521| 10023283211|
|f1727faa-de41-4c1...|    2021-11-26|  22450| 10023283211|
|4b6ba050-bb75-446...|    2021-11-26|   4380| 10023283211|
|a4b4af18-b067-4ac...|    2021-11-26|  16219| 10023283211|
|5cc7048f-bfea-4db...|    2021-11-26|  19326| 10023283211|
+--------------------+--------------+-------+------------+
only showing top 5 rows



In [11]:
trans_sdf = trans_sdf.join(consumer_fraud_sdf, on=['user_id', 'order_datetime'], how = 'left').withColumnRenamed('fraud_probability', 'consumer_fraud')

In [12]:
trans_sdf.show(5)

+-------+--------------+--------------------+------------+-----------+-----------------+
|user_id|order_datetime|            order_id|merchant_abn|consumer_id|   consumer_fraud|
+-------+--------------+--------------------+------------+-----------+-----------------+
|   2442|    2021-06-26|70b8d40f-1ec4-4be...| 10023283211|    1157533|9.611722331189984|
|   3224|    2021-05-05|0105037f-9620-475...| 10023283211|     741775|9.611722331189984|
|   4084|    2022-02-03|931c77f1-d707-49f...| 10023283211|    1213148|9.611722331189984|
|   4739|    2021-03-09|bf2a2a00-e9d3-447...| 10023283211|     225564|9.611722331189984|
|   6139|    2021-05-20|275af4b9-ea83-47b...| 10023283211|     188846|9.611722331189984|
+-------+--------------+--------------------+------------+-----------+-----------------+
only showing top 5 rows



In [13]:
merchant_fraud_sdf.show(5)

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 10023283211|    2021-03-01| 30.57903215900633|
| 10023283211|    2021-03-05| 30.57903215900633|
| 10023283211|    2021-03-06|29.563080709651327|
| 10023283211|    2021-03-15| 80.80054474543395|
| 10023283211|    2021-03-17| 30.57903215900633|
+------------+--------------+------------------+
only showing top 5 rows



In [14]:
trans_sdf = trans_sdf.join(merchant_fraud_sdf, on=['merchant_abn', 'order_datetime'], how = 'left').withColumnRenamed('fraud_probability', 'merchant_fraud')

In [15]:
trans_sdf.show(5)

+------------+--------------+-------+--------------------+-----------+-----------------+------------------+
|merchant_abn|order_datetime|user_id|            order_id|consumer_id|   consumer_fraud|    merchant_fraud|
+------------+--------------+-------+--------------------+-----------+-----------------+------------------+
| 10023283211|    2021-06-26|   2442|70b8d40f-1ec4-4be...|    1157533|9.611722331189984| 30.57903215900633|
| 10023283211|    2021-05-05|   3224|0105037f-9620-475...|     741775|9.611722331189984|29.563080709651327|
| 10023283211|    2022-02-03|   4084|931c77f1-d707-49f...|    1213148|9.611722331189984|29.563080709651327|
| 10023283211|    2021-03-09|   4739|bf2a2a00-e9d3-447...|     225564|9.611722331189984|29.563080709651327|
| 10023283211|    2021-05-20|   6139|275af4b9-ea83-47b...|     188846|9.611722331189984|29.563080709651327|
+------------+--------------+-------+--------------------+-----------+-----------------+------------------+
only showing top 5 rows



- prob of trans by daily fraud probs

In [16]:
trans_sdf = trans_sdf.withColumn('average_probability', ((trans_sdf.consumer_fraud + trans_sdf.merchant_fraud)/2))

In [17]:
trans_sdf.show(5)

+------------+--------------+-------+--------------------+-----------+-----------------+------------------+-------------------+
|merchant_abn|order_datetime|user_id|            order_id|consumer_id|   consumer_fraud|    merchant_fraud|average_probability|
+------------+--------------+-------+--------------------+-----------+-----------------+------------------+-------------------+
| 10023283211|    2021-06-26|   2442|70b8d40f-1ec4-4be...|    1157533|9.611722331189984| 30.57903215900633| 20.095377245098156|
| 10023283211|    2021-05-05|   3224|0105037f-9620-475...|     741775|9.611722331189984|29.563080709651327| 19.587401520420656|
| 10023283211|    2022-02-03|   4084|931c77f1-d707-49f...|    1213148|9.611722331189984|29.563080709651327| 19.587401520420656|
| 10023283211|    2021-03-09|   4739|bf2a2a00-e9d3-447...|     225564|9.611722331189984|29.563080709651327| 19.587401520420656|
| 10023283211|    2021-05-20|   6139|275af4b9-ea83-47b...|     188846|9.611722331189984|29.5630807096513

- calculate fraud prob on merchats

In [18]:
merchants = trans_sdf.groupBy('merchant_abn').agg(
    F.avg('average_probability').alias('fraud_prob'))
merchants.show(5)

+------------+------------------+
|merchant_abn|        fraud_prob|
+------------+------------------+
| 73256306726| 19.66385103484991|
| 83412691377|20.111579242679717|
| 24406529929| 20.13478111396258|
| 38700038932|26.792685232792838|
| 34440496342| 45.03580683409474|
+------------+------------------+
only showing top 5 rows



### Three categories based on life cycle

In [19]:
merchant_name = read_file(spark, 'filled.parquet', '../data/curated/')

|> Loading File...
|> Loading Finished!
-RECORD 0-------------------------------------------------------------------------------------------
 user_id         | 10285                                                                            
 merchant_abn    | 10023283211                                                                      
 dollar_value    | 311.8728694054186                                                                
 order_id        | e41d1fb7-d7b9-4758-9c20-1d34337a312d                                             
 merchant_name   | Felis Limited                                                                    
 tags            | furniture, home furnishings and equipment shops, and manufacturers, except ap... 
 take_rate       | 0.18                                                                             
 type            | e                                                                                
 postcode        | 6479                            

In [20]:
merchant_name = merchant_name.select('merchant_abn', 'merchant_name', 'tags', 'take_rate', 'type').drop_duplicates()

In [21]:
merchant_name.count()

4422

In [22]:
merchants = merchants.join(merchant_name, on=['merchant_abn'], how = 'left')

In [23]:
high_interval = ['antique shops - sales, repairs, and restoration services', 'art dealers and galleries', 'bicycle shops - sales and service', 'equipment, tool, furniture, and appliance rent al and leasing', 'health and beauty spas', 'hobby, toy and game shops', 'jewelry, watch, clock, and silverware shops', 'stationery, office supplies and printing and writing paper', 'telecom']

mid_interval = ['artist supply and craft shops', 'books, periodicals, and newspapers', 'computer programming , data processing, and integrated systems design services', 'furniture, home furnishings and equipment shops, and manufacturers, except appliances', 'lawn and garden supply outlets, including nurseries', 'motor vehicle supplies and new parts', 'music shops - musical instruments, pianos, and sheet music', 'shoe shops', ]

low_interval = ['cable, satellite, and other pay television and radio services', 'computers, computer peripheral equipment, and software', 'digital goods: books, movies, music', 'florists supplies, nursery stock, and flowers', 'gift, card, novelty, and souvenir shops', 'opticians, optical goods, and eyeglasses', 'tent and awning shops', 'watch, clock, and jewelry repair shops']

In [24]:
print(len(high_interval))
print(len(mid_interval))
print(len(low_interval))

9
8
8


In [25]:
merchants.count()

4422

In [26]:
merchant_df = merchants.to_pandas_on_spark()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyspark/sql/dataframe.py:3315: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  FutureWarning,


In [27]:
from collections import defaultdict
lifecycle_count = defaultdict()
for type in high_interval:
    lifecycle_count[type] = 0
    
for type in mid_interval:
    lifecycle_count[type] = 1

for type in low_interval:
    lifecycle_count[type] = 2

In [28]:
merchant_df['lifecycle_count'] = merchant_df.tags.apply(lambda x: lifecycle_count[x] )

In [29]:
merchant_group = merchant_df.to_spark()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [30]:
tag1 = merchant_group.filter(merchant_group.lifecycle_count == 0).orderBy('fraud_prob', ascending=False)

In [31]:
from pyspark.sql.window import Window
fraud_on_mercants = Window.partitionBy("lifecycle_count").orderBy("fraud_prob")
rank = merchant_group.withColumn("rank",F.rank().over(fraud_on_mercants))

In [32]:
rank.show(10)

+------------+------------------+--------------------+--------------------+-----------------+----+---------------+----+
|merchant_abn|        fraud_prob|       merchant_name|                tags|        take_rate|type|lifecycle_count|rank|
+------------+------------------+--------------------+--------------------+-----------------+----+---------------+----+
| 52266154043|13.827869164841578|Lobortis Tellus J...|art dealers and g...|             6.88|   a|              0|   1|
| 17175760321| 13.83522474132005|Morbi Metus Vivam...|hobby, toy and ga...|             6.33|   a|              0|   2|
| 15073921424|13.863470528314354|                null|             telecom|4.466589049638232|   b|              0|   3|
| 28717590335|13.877677438359955|                null|             telecom|4.400444888935293|   b|              0|   4|
| 85684792409| 13.91130688006743|Risus Donec Nibh LLC|health and beauty...|             6.37|   a|              0|   5|
| 27207000299| 13.91130688006743|Egestas

In [33]:
rank.filter(F.col('lifecycle_count')==0)

merchant_abn,fraud_prob,merchant_name,tags,take_rate,type,lifecycle_count,rank
52266154043,13.827869164841578,Lobortis Tellus J...,art dealers and g...,6.88,a,0,1
17175760321,13.83522474132005,Morbi Metus Vivam...,"hobby, toy and ga...",6.33,a,0,2
15073921424,13.863470528314354,null,telecom,4.466589049638232,b,0,3
28717590335,13.877677438359955,null,telecom,4.400444888935293,b,0,4
85684792409,13.91130688006743,Risus Donec Nibh LLC,health and beauty...,6.37,a,0,5
27207000299,13.91130688006743,Egestas Sed Phare...,bicycle shops - s...,5.84,a,0,5
20877828351,13.91130688006743,Semper Dui LLP,health and beauty...,4.98,b,0,5
78526643558,13.911306880067432,Arcu Institute,health and beauty...,6.29,a,0,8
39961396807,13.911306880067432,Sed Dolor Fusce LLP,"hobby, toy and ga...",5.53,a,0,8
25876985396,13.911306880067434,Vitae Erat Vivamu...,"stationery, offic...",4.19,b,0,10


In [34]:
rank.filter(F.col('lifecycle_count')==1)

merchant_abn,fraud_prob,merchant_name,tags,take_rate,type,lifecycle_count,rank
95856918319,13.903626047428073,Scelerisque Scele...,music shops - mus...,6.43,a,1,1
24314208971,13.91130688006743,Metus Ltd,computer programm...,5.73,a,1,2
87824427219,13.91130688006743,Phasellus Ferment...,artist supply and...,6.72,a,1,2
29917527785,13.911306880067432,Ullamcorper Corp.,"books, periodical...",4.1,b,1,4
27950649624,13.911306880067432,null,motor vehicle sup...,4.277697083403025,b,1,4
35194643826,13.911306880067432,Orci Industries,music shops - mus...,6.61,a,1,4
40751557554,13.911306880067432,Nisi Cum Industries,artist supply and...,4.48,b,1,4
68275485458,13.911306880067432,Morbi Sit Inc.,artist supply and...,5.62,a,1,4
20073297612,13.911306880067432,Dictum Sapien Ass...,artist supply and...,4.0,b,1,4
20415536115,13.911306880067434,Nibh Phasellus Nu...,lawn and garden s...,5.84,a,1,10


In [35]:
rank.filter(F.col('lifecycle_count')==2)

merchant_abn,fraud_prob,merchant_name,tags,take_rate,type,lifecycle_count,rank
43308864284,13.910174767241156,Eleifend Incorpor...,tent and awning s...,4.67,b,2,1
57798993346,13.91130688006743,Malesuada LLC,tent and awning s...,4.34,b,2,2
61569154734,13.91130688006743,Blandit At Nisi PC,tent and awning s...,5.78,a,2,2
60218700582,13.911306880067432,null,tent and awning s...,4.415571705599518,b,2,4
97748234104,13.911306880067432,Vitae Diam PC,florists supplies...,6.63,a,2,4
29681526553,13.911306880067432,Cras Vehicula Ali...,tent and awning s...,6.24,a,2,4
66650274915,13.911306880067432,Gravida Praesent ...,florists supplies...,6.3,a,2,4
45783462228,13.911306880067432,Pretium Neque Mor...,tent and awning s...,6.61,a,2,4
52379134298,13.911306880067436,Eu Foundation,"computers, comput...",6.53,a,2,9
68656010159,13.921709607218327,Suspendisse Limited,tent and awning s...,5.97,a,2,10


In [36]:
merchant_group.sort('fraud_prob')

merchant_abn,fraud_prob,merchant_name,tags,take_rate,type,lifecycle_count
52266154043,13.827869164841578,Lobortis Tellus J...,art dealers and g...,6.88,a,0
17175760321,13.83522474132005,Morbi Metus Vivam...,"hobby, toy and ga...",6.33,a,0
15073921424,13.863470528314354,null,telecom,4.466589049638232,b,0
28717590335,13.877677438359955,null,telecom,4.400444888935293,b,0
95856918319,13.903626047428073,Scelerisque Scele...,music shops - mus...,6.43,a,1
43308864284,13.910174767241156,Eleifend Incorpor...,tent and awning s...,4.67,b,2
20877828351,13.91130688006743,Semper Dui LLP,health and beauty...,4.98,b,0
85684792409,13.91130688006743,Risus Donec Nibh LLC,health and beauty...,6.37,a,0
57798993346,13.91130688006743,Malesuada LLC,tent and awning s...,4.34,b,2
87824427219,13.91130688006743,Phasellus Ferment...,artist supply and...,6.72,a,1
